In [6]:
import datetime as dt
import requests
import os
import time
from bs4 import BeautifulSoup
import markdownify as mf
from tqdm.notebook import tqdm

- 아무 신문사
  - 기사 목록 아무데나 들어가서 페이지 수 기준 10페이지만큼 컨텐츠를 수집하고 결과 스크린샷 찍어서 보내기

In [7]:
url_f = 'https://www.ytn.co.kr/news/list.php?page={0}&mcd=0102'

In [2]:
url = 'https://www.ytn.co.kr/news/list.php?page=1&mcd=0102'

In [3]:
session = requests.Session()

session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
})



In [4]:
try:
    r = session.get(url)
        
    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
except Exception as e:
    print("접속에 실패했습니다.")
    print(e)

r.encoding = "utf-8"
print(r.text)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="ko" xml:lang="ko">
<head>
<title>경제 목록</title>
<link rel="shortcut icon" href="/img/comm/favicon.ico" type="image/x-icon">
<link rel="icon" href="/img/comm/favicon.ico" type="image/x-icon">
<meta http-equiv="X-UA-Compatible" content="IE=edge" />
<meta http-equiv="content-type" content="text/html;charset=UTF-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1">
<meta name="format-detection" content="telephone=no">
<meta name="apple-mobile-web-app-capable" content="yes" />
<meta name="naver-site-verification" content="864bdc3e0e2612f9d3303eee5fdf570708f7328e"/>
<meta property="og:site_name" content="YTN"/>
<meta property="fb:pages" content="237776059583039">
<meta name='title' content='경제 목록'/>
<meta name='description' content='대한민국 24시간 뉴스 방송채널, 실시간 속보 및 제보하기, 경제'/>
<meta name='keywords' content='YTN

In [5]:
soup = BeautifulSoup(r.text)

In [11]:
len(soup.select('li.txt'))

17

In [13]:
a = soup.select('li.txt')

In [34]:
a[0].select('span.til')[0].text

'185개 품목 공급망 관리...2030년 특정국 의존도 50%↓'

In [26]:
a[0].select('span.til')[0].text

'185개 품목 공급망 관리...2030년 특정국 의존도 50%↓'

In [22]:
a[0].select('a')[0].attrs['href']

'https://www.ytn.co.kr/_ln/0102_202312131427381656'

In [27]:
a[0].select('span.date')[0].text

'2023-12-13 14:27'

In [35]:
def rip_list(url):
    try:
        r = session.get(url)
            
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)
    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)

    r.encoding = "utf-8"
    soup = BeautifulSoup(r.text)
    soup2 = soup.select('li.txt')
    posts = []

    for i in soup2:
        title = i.select('span.til')[0].text
        href = i.select('a')[0].attrs['href']
        post_date = i.select('span.date')[0].text

        posts.append({
            'title':title, 'link':href, 'date':post_date
        })
    
    return posts


In [37]:
all_posts = []

for j in range(1, 11):
    url2 = url_f.format(j)
    all_posts += rip_list(url2)

In [47]:
def rip_content(url):
    try:
        r = session.get(url)
                
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)
    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)

    r.encoding = "utf-8"
    soup = BeautifulSoup(r.text)
    domain = 'https://www.ytn.co.kr'
    content = soup.select('div.box > div.article > span')[0]

    body = str(content)
    rebody = body.replace('src="/', 'src="%s/' % domain)

    md = mf.markdownify(rebody)

    return md

In [68]:
dirname = dt.datetime.now().strftime('YTN_글_수집_%y%m%d_%H%M%S')

if not os.path.exists(dirname):
    os.mkdir(dirname)

progress = tqdm(total = len(all_posts))

for j in all_posts:
    md = rip_content(j['link'])

    fname = '%s_%s.md' % (j['date'], j['title'])
    fname = fname.replace(" ", "_").replace(".","").replace('md', '.md').replace('"', '').replace("?", "")
    fname = './' + dirname + '/' + fname
    print(fname)

    with open(fname, 'w', encoding = 'utf-8')as f:
        f.write(md)

    print(f"{fname}이 저장되었습니다.")
    progress.update()

  0%|          | 0/119 [00:00<?, ?it/s]

./YTN_글_수집_231213_160951/2023-12-13_14:27_185개_품목_공급망_관리2030년_특정국_의존도_50%↓.md
./YTN_글_수집_231213_160951/2023-12-13_14:27_185개_품목_공급망_관리2030년_특정국_의존도_50%↓.md이 저장되었습니다.
./YTN_글_수집_231213_160951/2023-12-13_14:26_초단기_예·적금,_신협_상품도_금리_비교_가능해진다.md
./YTN_글_수집_231213_160951/2023-12-13_14:26_초단기_예·적금,_신협_상품도_금리_비교_가능해진다.md이 저장되었습니다.
./YTN_글_수집_231213_160951/2023-12-13_14:26_금융당국_주담대_증가세_둔화이번_달_스트레스_DSR_발표.md
./YTN_글_수집_231213_160951/2023-12-13_14:26_금융당국_주담대_증가세_둔화이번_달_스트레스_DSR_발표.md이 저장되었습니다.
./YTN_글_수집_231213_160951/2023-12-13_14:01_중국산_김치에_국산_표시원산지표시_위반_132곳_적발.md
./YTN_글_수집_231213_160951/2023-12-13_14:01_중국산_김치에_국산_표시원산지표시_위반_132곳_적발.md이 저장되었습니다.
./YTN_글_수집_231213_160951/2023-12-13_13:44_지난달_전체_금융권_가계대출_26조↑증가_폭_줄어.md
./YTN_글_수집_231213_160951/2023-12-13_13:44_지난달_전체_금융권_가계대출_26조↑증가_폭_줄어.md이 저장되었습니다.
./YTN_글_수집_231213_160951/2023-12-13_13:12_새_아파트_인지세_공급업자·분양받는_사람_'반씩'_부담.md
./YTN_글_수집_231213_160951/2023-12-13_13:12_새_아파트_인지세_공급업자·분양받는_사람_'반씩'_부담.md이 저장되었습니다.
./YTN_글_수집_231213_160951/2023-12-1